In [45]:
#Basic Packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import confusion_matrix
import squarify
import ast
#To find addresses
import reverse_geocoder as rg
from geopy.geocoders import Nominatim

#Plot Maps
import folium
from folium.plugins import HeatMap, MarkerCluster 

#Functions
import Functions

%load_ext autoreload
%autoreload 2

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")


#Para fazer as regras de associação

import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
individuals = pd.read_csv('customers_clustered.csv')
basket = pd.read_csv('Customer Basket Dataset.csv')

In [47]:
individuals.set_index('customer_id', inplace=True)

In [48]:
basket['invoice_id'].nunique()

79749

In [49]:
basket.drop_duplicates(subset='invoice_id', inplace=True)

In [50]:
[ast.literal_eval(element) for element in list(basket['list_of_goods'])]

[['fresh bread',
  'cake',
  'grated cheese',
  'french fries',
  'whole wheat pasta',
  'candy bars',
  'beats headphones',
  'half-life 2',
  'phone car charger',
  'cream',
  'cereals',
  'babies food'],
 ['bramble',
  'beer',
  'white wine',
  'almonds',
  'half-life: alyx',
  'cauliflower',
  'cider',
  'yogurt cake',
  'champagne',
  'chili',
  'soda',
  'black beer',
  'catfish'],
 ['fromage blanc',
  'olive oil',
  'tomato sauce',
  'avocado',
  'megaman zero 2',
  'cooking oil',
  'iPad',
  'shampoo',
  'oil',
  'green tea'],
 ['pokemon shield',
  'white wine',
  'google tablet',
  'canned_tuna',
  'green beans',
  'bacon',
  'cider',
  'pokemon sword',
  'beats headphones',
  'airpods',
  'phone charger',
  'metroid prime'],
 ['olive oil',
  'deodorant',
  'whole weat flour',
  'chutney',
  'escalope',
  'gums',
  'rice',
  'airpods',
  'oil',
  'cookies',
  'muffins',
  'laptop'],
 ['ratchet & clank 2',
  'megaman zero',
  'eggplant',
  'body spray',
  'barbecue sauce',
  's

In [51]:
individuals

,customer_name,customer_gender,kids_home,teens_home,number_complaints,distinct_stores_visited,lifetime_spend_groceries,lifetime_spend_electronics,typical_hour,lifetime_spend_vegetables,...,total_lifetime_spend,minors_in_household,birthday,birthmonth,age,education,lifetime_spend_animal_protein,lifetime_spend_technology,cluster_kmeans3,cluster_kmeansZ
customer_id,,,,,,,,,,,,,,,,,,,,,
4239,Sara Moscowitz,1.0,0.0,0.0,0.0,2.0,91.0,20.0,22.0,22.0,...,1545.0,0.0,10.0,9.0,24.0,0.0,101.0,125.0,4,1
19330,Donald Robertson,0.0,1.0,0.0,0.0,2.0,197.0,5958.0,21.0,20.0,...,10219.0,1.0,26.0,6.0,42.0,0.0,166.0,7922.0,2,2
2272,Felicia Bradshaw,1.0,1.0,1.0,0.0,20.0,397.0,20.0,9.0,310.0,...,1753.0,2.0,31.0,1.0,36.0,3.0,374.0,25.0,5,4
25288,Emma Williams,1.0,3.0,4.0,1.0,2.0,5282.0,237.0,12.0,499.0,...,11195.0,7.0,17.0,6.0,81.0,0.0,2149.0,1315.0,0,5
12829,Hilda Huerta,1.0,0.0,0.0,0.0,2.0,82.0,22.0,21.0,9.0,...,1370.0,0.0,31.0,1.0,23.0,0.0,78.0,125.0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4689,Justin`s Supermarket,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6
16270,Melissa`s Supermarket,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6
10262,Francis`s Supermarket,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6


In [52]:
basket = basket.merge(individuals[['cluster_kmeans3', 'cluster_kmeansZ']], on='customer_id' )

In [53]:
basket.set_index('invoice_id', inplace=True)

In [55]:
te = TransactionEncoder()
basket_clusterZ0 = basket[basket['cluster_kmeansZ'] == 0]
basket_clusterZ0.drop(['customer_id', 'cluster_kmeans3', 'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ0 = [ast.literal_eval(element) for element in list(basket_clusterZ0['list_of_goods'])]
te_fit = te.fit(basket_clusterZ0).transform(basket_clusterZ0)
transactions_itemsZ0 = pd.DataFrame(te_fit, columns=te.columns_)

In [56]:
basket_clusterZ1 = basket[basket['cluster_kmeansZ'] == 1]
basket_clusterZ1.drop(['customer_id', 'cluster_kmeans3', 	'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ1 = [ast.literal_eval(element) for element in list(basket_clusterZ1['list_of_goods'])]
te_fit = te.fit(basket_clusterZ1).transform(basket_clusterZ1)
transactions_itemsZ1 = pd.DataFrame(te_fit, columns=te.columns_)

In [68]:
basket_clusterZ2 = basket[basket['cluster_kmeansZ'] == 2]
basket_clusterZ2.drop(['customer_id', 'cluster_kmeans3',	'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ2 = [ast.literal_eval(element) for element in list(basket_clusterZ2['list_of_goods'])]
te_fit = te.fit(basket_clusterZ2).transform(basket_clusterZ2)
transactions_itemsZ2 = pd.DataFrame(te_fit, columns=te.columns_)

In [76]:
basket_clusterZ3 = basket[basket['cluster_kmeansZ'] == 3]
basket_clusterZ3.drop(['customer_id', 'cluster_kmeans3',		'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ3 = [ast.literal_eval(element) for element in list(basket_clusterZ3['list_of_goods'])]
te_fit = te.fit(basket_clusterZ3).transform(basket_clusterZ3)
transactions_itemsZ3 = pd.DataFrame(te_fit, columns=te.columns_)

In [78]:
basket_clusterZ4 = basket[basket['cluster_kmeansZ'] == 4]
basket_clusterZ4.drop(['customer_id', 'cluster_kmeans3',		'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ4 = [ast.literal_eval(element) for element in list(basket_clusterZ4['list_of_goods'])]
te_fit = te.fit(basket_clusterZ4).transform(basket_clusterZ4)
transactions_itemsZ4 = pd.DataFrame(te_fit, columns=te.columns_)

In [82]:
basket_clusterZ5 = basket[basket['cluster_kmeansZ'] == 5]
basket_clusterZ5.drop(['customer_id', 'cluster_kmeans3', 	'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ5 = [ast.literal_eval(element) for element in list(basket_clusterZ5['list_of_goods'])]
te_fit = te.fit(basket_clusterZ5).transform(basket_clusterZ5)
transactions_itemsZ5 = pd.DataFrame(te_fit, columns=te.columns_)

In [83]:
basket_clusterZ6 = basket[basket['cluster_kmeansZ'] == 6]
basket_clusterZ6.drop(['customer_id', 'cluster_kmeans3',		'cluster_kmeansZ'], inplace=True, axis=1)
basket_clusterZ6 = [ast.literal_eval(element) for element in list(basket_clusterZ6['list_of_goods'])]
te_fit = te.fit(basket_clusterZ6).transform(basket_clusterZ6)
transactions_itemsZ6 = pd.DataFrame(te_fit, columns=te.columns_)

ASSOCIATION RULES PARA O CLUSTER 0 DA SOLUÇÃO Z

In [62]:
transactions_itemsZ0

,airpods,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,beats headphones,beer,...,vacuum cleaner,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9341,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9342,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9343,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9344,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [63]:
frequent_itemsets = apriori(transactions_itemsZ0, min_support=0.01, use_colnames=True)

In [64]:
frequent_itemsets

,support,itemsets
0,0.055425,(airpods)
1,0.021507,(almonds)
2,0.036486,(antioxydant juice)
3,0.748235,(asparagus)
4,0.172801,(avocado)
...,...,...
3146,0.010593,"(melons, zucchini, carrots, tomatoes, mashed p..."
3147,0.011342,"(cauliflower, carrots, tomatoes, frozen vegeta..."
3148,0.010486,"(green beans, frozen vegetables, carrots, toma..."
3149,0.014445,"(melons, carrots, tomatoes, frozen vegetables,..."


CLUSTER 0 É DOS VEGETARIANOS CONFIRMA -SE :)

In [65]:
rules_grocery = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

In [66]:
rules_grocery.sort_values(by = 'lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
9932,"(corn, tomatoes)","(carrots, melons)",0.042906,0.186711,0.010807,0.251870,1.348986,2.795734e-03,1.087096,0.270300
9937,"(carrots, melons)","(corn, tomatoes)",0.186711,0.042906,0.010807,0.057880,1.348986,2.795734e-03,1.015894,0.318094
9938,(corn),"(tomatoes, carrots, melons)",0.048898,0.164027,0.010807,0.221007,1.347376,2.786163e-03,1.073145,0.271071
9931,"(tomatoes, carrots, melons)",(corn),0.164027,0.048898,0.010807,0.065884,1.347376,2.786163e-03,1.018184,0.308403
13237,"(asparagus, tomatoes, shower gel)","(shallot, carrots)",0.083030,0.103895,0.011342,0.136598,1.314773,2.715354e-03,1.037877,0.261091
...,...,...,...,...,...,...,...,...,...,...
8321,"(ham, frozen vegetables, tomatoes)",(asparagus),0.015301,0.748235,0.011449,0.748252,1.000023,2.633154e-07,1.000068,0.000023
8324,(asparagus),"(ham, frozen vegetables, tomatoes)",0.748235,0.015301,0.011449,0.015301,1.000023,2.633154e-07,1.000000,0.000091
1232,(asparagus),"(cookies, carrots)",0.748235,0.015301,0.011449,0.015301,1.000023,2.633154e-07,1.000000,0.000091
6591,"(toothpaste, carrots)","(asparagus, frozen vegetables)",0.092232,0.198374,0.018297,0.198376,1.000011,2.060729e-07,1.000003,0.000012


CLUSTER 1 SOL Z

In [67]:
frequent_itemsets1 = apriori(transactions_itemsZ1, min_support=0.01, use_colnames=True)
rules_grocery1 = association_rules(frequent_itemsets1, metric='lift', min_threshold=1)
rules_grocery1.sort_values(by = 'lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6766,"(black beer, dessert wine)","(white wine, french wine)",0.042109,0.094696,0.010607,0.251897,2.660044,0.006619,1.210132,0.651500
6767,"(white wine, french wine)","(black beer, dessert wine)",0.094696,0.042109,0.010607,0.112011,2.660044,0.006619,1.078720,0.689345
6639,"(google tablet, beer)","(white wine, dessert wine)",0.022428,0.183387,0.010863,0.484330,2.641036,0.006750,1.583598,0.635616
6642,"(white wine, dessert wine)","(google tablet, beer)",0.183387,0.022428,0.010863,0.059233,2.641036,0.006750,1.039123,0.760899
6638,"(white wine, google tablet)","(beer, dessert wine)",0.050543,0.082875,0.010863,0.214918,2.593264,0.006674,1.168189,0.647092
...,...,...,...,...,...,...,...,...,...,...
6117,"(oil, cake, cooking oil)",(asparagus),0.155591,0.063706,0.010096,0.064887,1.018538,0.000184,1.001263,0.021554
1117,(gums),(shampoo),0.233099,0.044984,0.010607,0.045504,1.011568,0.000121,1.000545,0.014911
1116,(shampoo),(gums),0.044984,0.233099,0.010607,0.235795,1.011568,0.000121,1.003528,0.011974
937,(gums),(eggs),0.233099,0.054121,0.012652,0.054276,1.002862,0.000036,1.000164,0.003722


CLUSTER 2 SOL Z

In [72]:
frequent_itemsets2 = apriori(transactions_itemsZ2, min_support=0.01, use_colnames=True)
rules_grocery2 = association_rules(frequent_itemsets2, metric='lift', min_threshold=1)
rules_grocery2.sort_values(by = 'lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
14011,"(ratchet & clank, pokemon sword, pokemon violet)","(pokemon scarlet, bluetooth headphones)",0.130856,0.057283,0.010434,0.079740,1.392035,2.938612e-03,1.024403,0.324028
14026,"(pokemon scarlet, bluetooth headphones)","(ratchet & clank, pokemon sword, pokemon violet)",0.057283,0.130856,0.010434,0.182156,1.392035,2.938612e-03,1.062726,0.298740
14022,"(ratchet & clank, pokemon sword)","(pokemon violet, pokemon scarlet, bluetooth he...",0.260221,0.029280,0.010434,0.040098,1.369463,2.815066e-03,1.011270,0.364686
14015,"(pokemon violet, pokemon scarlet, bluetooth he...","(ratchet & clank, pokemon sword)",0.029280,0.260221,0.010434,0.356364,1.369463,2.815066e-03,1.149373,0.277924
14019,"(ratchet & clank, pokemon violet)","(pokemon scarlet, pokemon sword, bluetooth hea...",0.146614,0.052172,0.010434,0.071169,1.364125,2.785250e-03,1.020453,0.312789
...,...,...,...,...,...,...,...,...,...,...
9233,(pokemon sword),"(final fantasy XX, pokemon shield, pokemon vio...",0.876597,0.016397,0.014374,0.016397,1.000030,4.307918e-07,1.000000,0.000243
16661,"(ratchet & clank, pokemon sword, pokemon shiel...",(pokemon violet),0.027044,0.496060,0.013416,0.496063,1.000005,6.801976e-08,1.000005,0.000005
16672,(pokemon violet),"(ratchet & clank, pokemon sword, pokemon shiel...",0.496060,0.027044,0.013416,0.027044,1.000005,6.801976e-08,1.000000,0.000010
873,(pokemon shield),(white wine),0.653003,0.340290,0.222210,0.340290,1.000002,4.081185e-07,1.000001,0.000005


CLUSTER 3 SOL Z

In [77]:
frequent_itemsets3 = apriori(transactions_itemsZ3, min_support=0.01, use_colnames=True)
rules_grocery3 = association_rules(frequent_itemsets3, metric='lift', min_threshold=1)
rules_grocery3.sort_values(by = 'lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
7730,(olive oil),"(cooking oil, cake, yogurt cake)",0.165775,0.049607,0.010302,0.062142,1.252671,2.077878e-03,1.013365,2.417878e-01
7719,"(cooking oil, cake, yogurt cake)",(olive oil),0.049607,0.165775,0.010302,0.207661,1.252671,2.077878e-03,1.052864,2.122339e-01
5279,"(oil, milk)",(sandwich),0.109466,0.076762,0.010402,0.095021,1.237867,1.998752e-03,1.020176,2.157795e-01
5282,(sandwich),"(oil, milk)",0.076762,0.109466,0.010402,0.135505,1.237867,1.998752e-03,1.030120,2.081357e-01
5278,"(oil, sandwich)",(milk),0.067610,0.124369,0.010402,0.153846,1.237017,1.992978e-03,1.034837,2.054975e-01
...,...,...,...,...,...,...,...,...,...,...
7237,"(cake, gums)","(soup, cooking oil)",0.179927,0.153973,0.027704,0.153974,1.000009,2.400720e-07,1.000002,1.056681e-05
13501,"(muffins, napkins, gums)","(oil, candy bars)",0.027254,0.368805,0.010052,0.368807,1.000005,5.501650e-08,1.000003,5.626811e-06
13504,"(oil, candy bars)","(muffins, napkins, gums)",0.368805,0.027254,0.010052,0.027254,1.000005,5.501650e-08,1.000000,8.671584e-06
10812,"(muffins, olive oil)","(oil, cooking oil)",0.050608,0.576086,0.029154,0.576087,1.000001,2.750825e-08,1.000001,9.938333e-07


CLUSTER 4 SOL Z

In [81]:
frequent_itemsets4 = apriori(transactions_itemsZ4, min_support=0.01, use_colnames=True)
rules_grocery4 = association_rules(frequent_itemsets4, metric='lift', min_threshold=1)
rules_grocery4.sort_values(by = 'lift', ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10089,"(oil, chicken)","(muffins, cooking oil)",0.040264,0.200101,0.010446,0.259446,1.296572,0.002389,1.080135,0.238331
10092,"(muffins, cooking oil)","(oil, chicken)",0.200101,0.040264,0.010446,0.052205,1.296572,0.002389,1.012599,0.285955
5663,"(yams, oil)",(muffins),0.025659,0.302535,0.010041,0.391304,1.293416,0.002278,1.145835,0.232828
5666,(muffins),"(yams, oil)",0.302535,0.025659,0.010041,0.033188,1.293416,0.002278,1.007787,0.325255
10096,(chicken),"(muffins, oil, cooking oil)",0.045538,0.178296,0.010446,0.229399,1.286616,0.002327,1.066315,0.233396
10085,"(muffins, oil, cooking oil)",(chicken),0.178296,0.045538,0.010446,0.058589,1.286616,0.002327,1.013864,0.271104
2715,(candy bars),"(oil, cider)",0.415416,0.019371,0.010345,0.024902,1.285535,0.002298,1.005672,0.379951
2710,"(oil, cider)",(candy bars),0.019371,0.415416,0.010345,0.534031,1.285535,0.002298,1.254557,0.226501
3556,(chicken),"(muffins, cooking oil)",0.045538,0.200101,0.011663,0.256125,1.279975,0.002551,1.075313,0.229170
3553,"(muffins, cooking oil)",(chicken),0.200101,0.045538,0.011663,0.058287,1.279975,0.002551,1.013538,0.273453


CLUSTER 5 SOL Z

In [84]:
frequent_itemsets5 = apriori(transactions_itemsZ5, min_support=0.01, use_colnames=True)
rules_grocery5 = association_rules(frequent_itemsets5, metric='lift', min_threshold=1)
rules_grocery5.sort_values(by = 'lift', ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
14370,"(candy bars, ketchup)","(babies food, soup, cooking oil)",0.075579,0.136550,0.013825,0.182916,1.339552,0.003504,1.056746,0.274206
14351,"(babies food, soup, cooking oil)","(candy bars, ketchup)",0.136550,0.075579,0.013825,0.101242,1.339552,0.003504,1.028554,0.293569
14360,"(babies food, candy bars, ketchup)","(soup, cooking oil)",0.068471,0.154679,0.013825,0.201905,1.305316,0.003234,1.059173,0.251095
14361,"(soup, cooking oil)","(babies food, candy bars, ketchup)",0.154679,0.068471,0.013825,0.089376,1.305316,0.003234,1.022957,0.276702
9059,"(escalope, cooking oil)","(babies food, gums)",0.028562,0.307401,0.011412,0.399543,1.299745,0.002632,1.153453,0.237399
9058,"(babies food, gums)","(escalope, cooking oil)",0.307401,0.028562,0.011412,0.037123,1.299745,0.002632,1.008891,0.332975
9057,"(babies food, escalope)","(gums, cooking oil)",0.038539,0.228758,0.011412,0.296108,1.294419,0.002596,1.095683,0.236570
9060,"(gums, cooking oil)","(babies food, escalope)",0.228758,0.038539,0.011412,0.049886,1.294419,0.002596,1.011942,0.294917
9052,"(babies food, escalope, cooking oil)",(gums),0.025302,0.349332,0.011412,0.451031,1.291125,0.002573,1.185255,0.231335
9065,(gums),"(babies food, escalope, cooking oil)",0.349332,0.025302,0.011412,0.032668,1.291125,0.002573,1.007615,0.346539


cluster 6 sol z

In [85]:
frequent_itemsets6 = apriori(transactions_itemsZ6, min_support=0.01, use_colnames=True)
rules_grocery6 = association_rules(frequent_itemsets6, metric='lift', min_threshold=1)
rules_grocery6.sort_values(by = 'lift', ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
109056,"(butter, whole weat flour, candy bars)","(oil, cake, bluetooth headphones)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
126618,"(oil, cake, cereals, gums, cooking oil)","(french fries, mineral water)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
68914,"(butter, cake, deodorant, candy bars)",(bluetooth headphones),0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
69083,"(cake, bluetooth headphones)","(oil, butter, whole weat flour)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
69698,"(muffins, cake, melons)","(body spray, frozen vegetables)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
125664,"(butter, whole weat flour, deodorant, candy bars)","(oil, cake, bluetooth headphones)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
125422,"(oil, soup, burgers)","(cooking oil, asparagus, gums, cream)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
74740,"(meatballs, milk)","(cake, gluten free bar, candy bars)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
106384,"(eggs, cake, cooking oil)","(oil, candy bars, almonds)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
104434,"(napkins, olive oil, fromage blanc)","(shampoo, soup)",0.011834,0.011834,0.011834,1.0,84.5,0.011694,inf,1.0
